### Notebook for testing :

>ARIMA, 
>ARIMAX, 
>SARIMAX, 
>Smoothing exp and
>Winterhalt

In [7]:
import pandas as pd
import numpy as np 

import xgboost as xgb
import seaborn as sns
sns.set(style="darkgrid")
import matplotlib.pyplot as plt
sns.set(rc={'figure.figsize':(11.7,8.27)})

'''Predefined Functions for EDA'''
def eda(data):
    print("----------Top-5- Record----------")
    print(data.head(5))
    print("-----------Information-----------")
    print(data.info())
    print("-----------Data Types-----------")
    print(data.dtypes)
    print("----------Missing value-----------")
    print(data.isnull().sum())
    print("----------Null value-----------")
    print(data.isna().sum())
    print("----------Shape of Data----------")
    print(data.shape)
def graph_insight(data):
    print(set(data.dtypes.tolist()))
    df_num = data.select_dtypes(include = ['float64', 'int64'])
    df_num.hist(figsize=(16, 16), bins=50, xlabelsize=8, ylabelsize=8);
def drop_duplicate(data, subset):
    print('Before drop shape:', data.shape)
    before = data.shape[0]
    data.drop_duplicates(subset,keep='first', inplace=True) #subset is list where you have to put all column for duplicate check
    data.reset_index(drop=True, inplace=True)
    print('After drop shape:', data.shape)
    after = data.shape[0]
    print('Total Duplicate:', before-after)


In [29]:
shops=pd.read_csv('data/predict_future_sales/shops.csv')
train=pd.read_csv('data/predict_future_sales/sales_train.csv')
items=pd.read_csv('data/predict_future_sales/items.csv')
items_categories=pd.read_csv('data/predict_future_sales/item_categories.csv')
test=pd.read_csv('data/predict_future_sales/test.csv')
sample=pd.read_csv('data/predict_future_sales/sample_submission.csv')
items_categories['item_category_name']=items_categories['item_category_name'].astype(str)

In [30]:
items_categories.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [31]:
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [32]:
items_categories['meta-category']=items_categories['item_category_name'].apply(lambda x : x.split("-")[0])

In [33]:
len(items_categories['meta-category'].unique())

21

In [34]:
items_categories.shape

(84, 3)

In [35]:
shops['town']=shops['shop_name'].apply(lambda x : x.split(" ")[0])

In [36]:
len(shops['town'].unique())

32

In [37]:
shops.shape

(60, 3)

##### After grouping the category, we reduced the number from 84 to 21 and shops to half

In [38]:
train['date']=pd.to_datetime(train.date,format='%d.%m.%Y')
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-01-02,0,59,22154,999.00,1.0
1,2013-01-03,0,25,2552,899.00,1.0
2,2013-01-05,0,25,2552,899.00,-1.0
3,2013-01-06,0,25,2554,1709.05,1.0
4,2013-01-15,0,25,2555,1099.00,1.0


In [39]:
train=train[~train.duplicated()]
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-01-02,0,59,22154,999.00,1.0
1,2013-01-03,0,25,2552,899.00,1.0
2,2013-01-05,0,25,2552,899.00,-1.0
3,2013-01-06,0,25,2554,1709.05,1.0
4,2013-01-15,0,25,2555,1099.00,1.0


In [40]:
train = train[(train.item_price > 0) & (train.item_price < 300000)]


In [41]:
train.date=train.date.apply(lambda x: x.strftime('%Y-%m'))

In [42]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-01,0,59,22154,999.00,1.0
1,2013-01,0,25,2552,899.00,1.0
2,2013-01,0,25,2552,899.00,-1.0
3,2013-01,0,25,2554,1709.05,1.0
4,2013-01,0,25,2555,1099.00,1.0


In [43]:
df = train.groupby(['date','item_id','shop_id']).agg({'item_cnt_day':sum,
                                                     'item_price': 'mean'}).reset_index()
df = df[['date','item_id','shop_id','item_cnt_day']]
df = df.pivot_table(index=['item_id','shop_id'], columns='date',values='item_cnt_day',fill_value=0).reset_index()
df.head()


date,item_id,shop_id,2013-01,2013-02,2013-03,2013-04,2013-05,2013-06,2013-07,2013-08,...,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10
0,0,54,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,55,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,54,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,54,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,54,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
df[df['item_id']==50]

date,item_id,shop_id,2013-01,2013-02,2013-03,2013-04,2013-05,2013-06,2013-07,2013-08,...,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10
671,50,25,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
672,50,54,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
df_test = pd.merge(test, df, on=['item_id','shop_id'], how='left')
df_test = df_test.fillna(0)
df_test.head()

,ID,shop_id,item_id,2013-01,2013-02,2013-03,2013-04,2013-05,2013-06,2013-07,...,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10
0,0,5,5037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,1.0,0.0
1,1,5,5320,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,5,5233,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,2.0,0.0,1.0,3.0,1.0
3,3,5,5232,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,4,5,5268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
## bring the metacategory to the item table 
from sklearn import preprocessing


items = pd.merge(items,items_categories, on=['item_category_id'], how='left')
items=items[['item_id','meta-category']]
le = preprocessing.LabelEncoder()
le.fit(items['meta-category'])
items['meta-category']=le.transform(items['meta-category'])
le.fit(shops['town'])
shops['town']=le.transform(shops['town'])

items.head()

In [47]:
le.fit(shops['town'])

shops['town']=le.transform(shops['town'])



In [48]:
df_train = pd.merge( df,items, on=['item_id'], how='left')
df_train.head()

,item_id,shop_id,2013-01,2013-02,2013-03,2013-04,2013-05,2013-06,2013-07,2013-08,...,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,meta-category
0,0,54,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,11
1,1,55,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,15
2,2,54,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,11
3,3,54,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,11
4,4,54,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,11


In [49]:
df_test = pd.merge( df_test,items, on=['item_id'], how='left')
df_test.head()

,ID,shop_id,item_id,2013-01,2013-02,2013-03,2013-04,2013-05,2013-06,2013-07,...,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,meta-category
0,0,5,5037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,3.0,1.0,0.0,5
1,1,5,5320,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13
2,2,5,5233,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,2.0,0.0,1.0,3.0,1.0,5
3,3,5,5232,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,5
4,4,5,5268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5


In [50]:
shops=shops[['shop_id','town']]
df_train = pd.merge( df_train,shops, on=['shop_id'], how='left') 
df_test = pd.merge( df_test,shops, on=['shop_id'], how='left')
df_train.head()

,item_id,shop_id,2013-01,2013-02,2013-03,2013-04,2013-05,2013-06,2013-07,2013-08,...,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,meta-category,town
0,0,54,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,11,27
1,1,55,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,15,28
2,2,54,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,11,27
3,3,54,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,11,27
4,4,54,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,11,27


In [ ]:
df_train.drop(columns=['2013-01', '2013-02', '2013-03', '2013-04','2013-05', '2013-06', '2013-07', '2013-08'],inplace=True)
df_test.drop(columns=['2013-01', '2013-02', '2013-03', '2013-04','2013-05', '2013-06', '2013-07', '2013-08'],inplace=True)

In [68]:
gbm=xgb.XGBRegressor().fit(df_train.drop(columns=['2015-10']).values,df_train['2015-10'].values)

In [69]:
y_pred=gbm.predict(df_test.drop(columns=['2013-09','ID']).values)
y_pred

array([0.39668614, 0.0874013 , 0.6294452 , ..., 0.0874013 , 0.0874013 ,
       0.0874013 ], dtype=float32)

In [70]:
## xgboost submit 
preds = pd.DataFrame(y_pred.clip(0., 20.), columns=['item_cnt_month'])
preds.to_csv('submission.csv',index_label='ID')

In [73]:
preds


,item_cnt_month
0,0.396686
1,0.087401
2,0.629445
3,0.101577
4,0.087401
5,0.385901
6,0.882780
7,0.087401
8,0.672216
9,0.087401
